In [2]:
import pandas as pd
import mysql.connector

In [3]:
# připojit k databázi - db object pro referenci na připojení
db = mysql.connector.connect(
  host="localhost", # server - u nás na PC localhost, jinak např. https://svatky.adresa.info/
  user="root", # přihlášení
  password="M0ncouer",
  database="vs_zkouska" # výběr schématu/db -> odpovídá sql: use nase_db;
)
# cursor -> object, přes který provádím sql příkazy pomocí execute
cursor = db.cursor()

In [4]:
sql = f"show columns from predmet;"
cursor.execute(sql)
vysledok = cursor.fetchall()

In [5]:
vysledok

[('idpredmet', 'int', 'NO', 'PRI', None, 'auto_increment'),
 ('nazev', 'varchar(45)', 'YES', '', None, ''),
 ('obor', 'varchar(45)', 'YES', '', None, ''),
 ('ucitel_iducitel', 'int', 'NO', 'MUL', None, '')]

In [6]:
#Naplniť tabulky dátami
from faker import Faker
from random import randint

In [ ]:
"""
fake = Faker()
kolko = 100
ludia = []
for i in range(kolko):
    full_name = fake.name()
    meno = full_name.split(" ")[0]
    priezvisko = full_name.split(" ")[-1]
    vek = randint(19, 28)
    ludia.append([meno, priezvisko, vek])
"""

In [7]:
def generator_ludi(pocet, vek_od=18, vek_do=28):
    """
    vezme na vstupu počet lidí k vygenerování a vrátí list, kde jednotlivé prvky jsou listy se 3 hodnotami
    a to jméno, příjmení a věk
    """
    # z knihovny faker vytvoř generátor náhodných lidí
    fake = Faker()
    # list pro ukládání lidí
    ludia = []
    # zopakuj pocet krát generování člověka
    for i in range(pocet):
        # dej mi plné jméno náhodného člověka
        full_name = fake.name()
        # plné jméno je ve formátu 'meno priezvisko' -> rozděl text podle mezery, kdy první hodnota je jméno a druhá příjmení
        meno = full_name.split(" ")[0]
        priezvisko = full_name.split(" ")[-1]
        # vygeneruj náhodný vek
        vek = randint(vek_od, vek_do)
        # přidej do listu lidé všechny informace jako list
        ludia.append([meno, priezvisko, vek])
    return ludia

In [7]:
generator_ludi(10,18,45)

[['Mrs.', 'Bryan', 23],
 ['Megan', 'Parker', 39],
 ['Kathleen', 'Torres', 45],
 ['Tracy', 'Williams', 29],
 ['Natasha', 'Cisneros', 28],
 ['Richard', 'Baxter', 36],
 ['Amy', 'Snyder', 29],
 ['Sharon', 'Vaughn', 36],
 ['Heather', 'Norris', 37],
 ['Victor', 'Blankenship', 24]]

In [8]:
studenti = tuple(generator_ludi(100))

sql = "INSERT INTO student (jmeno, prijmeni, vek) VALUES "

# constructing sql statment with multiple row values
for row in studenti:
    sql += "('{}','{}','{}'),".format(row[0], row[1], row[2])

sql = sql[:-1]

try:
  cursor.execute(sql)
  db.commit()
  print("Úspěch")
except Exception as error:
  print(f"Error: {error}")

Úspěch


In [ ]:
studenti

In [11]:
ucitele = tuple(generator_ludi(10, 25, 80))

sql = "INSERT INTO ucitel (jmeno, prijmeni, vek) VALUES "

# constructing sql statment with multiple row values
for row in ucitele:
    sql += "('{}','{}','{}'),".format(row[0], row[1], row[2])

sql = sql[:-1]

try:
  cursor.execute(sql)
  db.commit()
  print("Úspěch")
except Exception as error:
  print(f"Error: {error}")

Úspěch


In [24]:
def zobraz_studentov(cursor):
    sql1 = f"select * from student;"
    cursor.execute(sql1)
    return cursor.fetchall()

In [31]:
sql2 = "DESCRIBE student;"
cursor.execute(sql2)
info_stlpce = cursor.fetchall()
#extrakcia pomocou list comprehension
nazvy_stlpcov = [col[0]for col in info_stlpce]

In [35]:
data = zobraz_studentov(cursor)

In [36]:
df_studenti = pd.DataFrame.from_records(data, columns=nazvy_stlpcov)

In [37]:
df_studenti

,idstudent,jmeno,prijmeni,vek
0,104,Kevin,Lozano,21
1,105,Kyle,Hernandez,23
2,106,Jennifer,Davis,18
3,107,Kayla,Estes,24
4,108,Trevor,Thompson,18
...,...,...,...,...
95,199,Anthony,Weber,27
96,200,Keith,Jenkins,26
97,201,Cathy,Sampson,23
98,202,Thomas,Winters,24


In [33]:
while cursor.nextset():
    pass

In [43]:
mistnosti = []
for i in range(3):
    zeme = input("Zadej zemi: ")
    mesto = input("Zadej město: ")
    adresa = input("Zadej adresu: ")
    ucebna = input("Zadej název učebny: ")
    mistnosti.append((zeme, mesto, adresa, ucebna))

In [44]:
mistnosti

[('SK', 'Bratislava', 'Lafranconi', '3DC'),
 ('SK', 'Žilina', 'Hlavná', '8C0'),
 ('SK', 'Ružomberok', 'Hanulu', 'C5C')]

In [45]:
sql = "INSERT INTO misto (zeme, mesto, adresa, ucebna) VALUES "

# constructing sql statment with multiple row values
for row in mistnosti:
    sql += "('{}','{}','{}','{}'),".format(row[0], row[1], row[2], row[3])

sql = sql[:-1]

try:
  cursor.execute(sql)
  db.commit()
  print("Úspěch")
except Exception as error:
  print(f"Error: {error}")

Úspěch


In [40]:
sql = f"select iducitel from ucitel;"
cursor.execute(sql)
vysledek = cursor.fetchall()

ucitelska_id = [prvek[0] for prvek in vysledek]

In [41]:
ucitelska_id

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [46]:
from random import choice

In [59]:
predmety = []
for i in range(3):
    nazev = input("Zadej nazev: ")
    obor = input("Zadej obor: ")
    ucitel_iducitel = choice(ucitelska_id)
    predmety.append((nazev, obor, ucitel_iducitel))

In [60]:
sql = "INSERT INTO predmet (nazev, obor, ucitel_iducitel) VALUES "

# constructing sql statment with multiple row values
for row in predmety:
    sql += "('{}','{}','{}'),".format(row[0], row[1], row[2])

sql = sql[:-1]

try:
  cursor.execute(sql)
  db.commit()
  print("Úspěch")
except Exception as error:
  print(f"Error: {error}") 

Úspěch


In [49]:
sql = f"select iducitel from ucitel;"
cursor.execute(sql)
vysledek = cursor.fetchall()

ucitel_id = [prvek[0] for prvek in vysledek]

sql = f"select idstudent from student;"
cursor.execute(sql)
vysledek = cursor.fetchall()

student_id = [prvek[0] for prvek in vysledek]

sql = f"select idpredmet from predmet;"
cursor.execute(sql)
vysledek = cursor.fetchall()

predmet_id = [prvek[0] for prvek in vysledek]

sql = f"select idmisto from misto;"
cursor.execute(sql)
vysledek = cursor.fetchall()

misto_id = [prvek[0] for prvek in vysledek]

In [61]:
zkousky = []
for i in range(3):
    termin = input("Zadej termín ve formátu '2024-10-05': ")
    online = input("Bude online: ")
    ucitel_iducitel = choice(ucitel_id)
    misto_idmisto = choice(misto_id)
    student_idstudent = choice(student_id)
    predmet_idpredmet = choice(predmet_id)
    zkousky.append((termin, online, ucitel_iducitel, misto_idmisto, student_idstudent, predmet_idpredmet))

In [62]:
sql = "INSERT INTO zkouska (termin, online, ucitel_iducitel, misto_idmisto, student_idstudent, predmet_idpredmet) VALUES "

# constructing sql statment with multiple row values
for row in zkousky:
    sql += "('{}','{}','{}','{}','{}','{}'),".format(row[0], row[1], row[2], row[3], row[4], row[5])

sql = sql[:-1]

try:
  cursor.execute(sql)
  db.commit()
  print("Úspěch")
except Exception as error:
  print(f"Error: {error}")

Úspěch


In [54]:
sql = f"select idzkouska from zkouska;"
cursor.execute(sql)
vysledek = cursor.fetchall()

zkouska_id = [prvek[0] for prvek in vysledek]

In [63]:
vysledky = []
for i in range(3):
    prospel = input("Zadej prospel: ")
    body = input("Zadej body: ")
    komentar = input("Zadej komentar: ")
    zkouska_idzkouska = choice(zkouska_id)
    vysledky.append((prospel, body, komentar, zkouska_idzkouska))

In [64]:
sql = "INSERT INTO vysledek (prospel, body, komentar, zkouska_idzkouska) VALUES "

# constructing sql statment with multiple row values
for row in vysledky:
    sql += "('{}','{}','{}','{}'),".format(row[0], row[1], row[2], row[3])

sql = sql[:-1]

try:
  cursor.execute(sql)
  db.commit()
  print("Úspěch")
except Exception as error:
  print(f"Error: {error}")

Úspěch
